In [1]:
using Plots
using LaTeXStrings
using Roots

# Examen 2
## Física computacional
#### Aquino Chapa Martha Abril 
Hecho en Julia 

#### 1. Se tiene un oscilador armónico en dos dimensiones (xy) en el que la fuerza está dada por $\vec{F}= −k\vec{r}$ y se tiene una fricción $\vec{f}_{fric} = −γ\vec{v}$ si $|\vec{v}| ≤ 1$, pero cuando $|\vec{v}| > 1$ la fricción es $\vec{f}_{fric} = −µv^{3/2}\hat{v}$. Si m = 1, k = 1, γ = 0.1 y µ = 0.2, resuelva las siguientes ejercicios:

#### (a) Encuentre las ecuaciones de movimiento en coordenadas cartesianas. No es necesario resolver las ecuaciones.

Debido a la segunda ley de Newton, tendremos que la fuerza total del sistema será igual a la masa por la aceleración del sistema. Así se tiene que:
$$m\vec{a}=\vec{F}+\vec{f}_{fric}=m\frac{d^{2}\vec{r}}{dt^{2}} $$

Por lo cual se obtiene que:
Si $|\vec{v}|≤1$
$$m\frac{d^{2}\vec{r}}{dt^{2}} =−k\vec{r}−γ\vec{v}=−k\vec{r}−γ\frac{d\vec{r}}{dt} $$
Si $|\vec{v}|>1$
$$m\frac{d^{2}\vec{r}}{dt^{2}} =−k\vec{r}−µv^{3/2}\hat{v}=−k\vec{r}−µv^{1/2}\vec{v}=−k\vec{r}−µ(\frac{dr}{dt})^{1/2}\frac{d\vec{r}}{dt}$$

Ahora, sustituimos los valores dados y separamos las ecuaciones por componentes cartesianas.

En la coordenada $x$:

$$ \begin{cases}
 m\frac{d^{2}x}{dt^{2}}=−kx−γ\frac{dx}{dt} &   si  & |\vec{v}|≤1 \\
m\frac{d^{2}x}{dt^{2}}=−kx−µ\left(\left(\frac{dx}{dt}\right)^{2}+\left(\frac{dy}{dt}\right)\right)^{1/4}\frac{dx}{dt} &  si  & |\vec{v}|>1
\end{cases}$$

En el eje y:
$$ \begin{cases}
 m\frac{d^{2}y}{dt^{2}}=−ky−γ\frac{dy}{dt} &   si  & |\vec{v}|≤1 \\
m\frac{d^{2}y}{dt^{2}}=−ky−µ\left(\left(\frac{dx}{dt}\right)^{2}+\left(\frac{dy}{dt}\right)\right)^{1/4}\frac{dy}{dt} &  si  & |\vec{v}|>1
\end{cases}$$

Sustituyendo las constantes dadas:

En el eje x:
$$ \begin{cases}
 \frac{d^{2}x}{dt^{2}}=−x−0.1\frac{dx}{dt} &   si  & |\vec{v}|≤1 \\
\frac{d^{2}x}{dt^{2}}=−x−0.2\left(\left(\frac{dx}{dt}\right)^{2}+\left(\frac{dy}{dt}\right)\right)^{1/4}\frac{dx}{dt} &  si  & |\vec{v}|>1
\end{cases}$$
En el eje y:
$$ \begin{cases}
 \frac{d^{2}y}{dt^{2}}=−y−0.1\frac{dy}{dt} &   si  & |\vec{v}|≤1 \\
\frac{d^{2}y}{dt^{2}}=−y−0.2\left(\left(\frac{dx}{dt}\right)^{2}+\left(\frac{dy}{dt}\right)\right)^{1/4}\frac{dy}{dt} &  si  & |\vec{v}|>1
\end{cases}$$


Con lo cual podemos escribir el siguiente sistema de ecuaciones diferenciales:

$$x'=v_x $$
$$y'=v_y$$
$$ \begin{cases}
 v_x'=−x−0.1v_x &   si  & |\vec{v}|≤1 \\
v_x'=−x−0.2\left(v_x^{2}+v_y^{2}\right)^{1/4}v_x &  si  & |\vec{v}|>1
\end{cases}$$
En el eje y:
$$ \begin{cases}
v'_y=−y−0.1v_y &   si  & |\vec{v}|≤1 \\
v_y'=−y−0.2\left(v_x^{2}+v_y^{2}\right)^{1/4}v_y &  si  & |\vec{v}|>1
\end{cases}$$

#### (b) Utilice el método de Runge-Kutta de 4to orden para encontrar la solución a las ecuaciones de movimiento en el inciso anterior con condiciones iniciales $\vec{r}_0 = (0, 1)$ y $\vec{v}_0 = (\cos θ,\sin θ)$, donde $\theta =\frac{\pi n}{6}$ con n = 0, 1, 2, 3, ..., 10, 11.

Con las ecuaciones anteriores, podemos crear una función que nos devuelva la información del sistema de ecuaciones diferenciales lineales.

In [ ]:
function sistema1(x,t)
    dx=x[3]
    dy=x[4]
    v=x[3]^2+x[4]^2
    if sqrt(v)<=1
        dv_x=-x[1]-0.1*x[3]
        dv_y=-x[2]-0.1*x[4]
    else
        dv_x=-x[1]-0.2*(v^(1/4))*x[3]
        dv_y=-x[2]-0.2*(v^(1/4))*x[4]
    end
    return [dx,dy,dv_x,dv_y]
end

Ahora, extraemos la función de Runge-Kutta de 4to orden visto en clase:

In [ ]:
function RK_4(edo,x_ini,t)
    sol = zeros( length(t) , length(x_ini) )
    sol[1,:] .= x_ini
    δ = t[2]-t[1]
    for i in 1:length(t)-1
        k1 = edo(sol[i,:],t[i])
        k2 = edo(sol[i,:] .+ 0.5*δ.*k1 , t[i] + 0.5*δ)
        k3 = edo(sol[i,:] .+ 0.5*δ.*k2 , t[i] + 0.5*δ)
        k4 = edo(sol[i,:] .+ δ.*k2 , t[i] + δ)
        sol[i+1,:] .= sol[i,:] .+ (δ/6.0).*(k1 .+ 2.0.*k2 .+ 2.0.*k3 .+ k4)
    end
    return sol
end

Probamos nuestra solución con n=0.

In [ ]:
tiempo1_p=collect(0:0.01:70π)
sol1_p=RK_4(sistema1,[0.0,1.0,1.0,0.0],tiempo1_p)

Ahora, creamos un ciclo para cumplir la condición de que $\theta=\frac{\pi n}{6}$ con n = 0, 1, 2, 3, ..., 10, 11.

In [ ]:
#Definimos nuestro conjunto de n's
n=collect(0:11)
#Definimos el tiempo donde evaluaremos la función
tiempo1=collect(0:0.1:100)
for i in n 
    sol1=RK_4(sistema1,[0.0,1.0,cos(i*π/6),sin(i*π/6)],tiempo1)
end

Presentamos el resultado a manera de gráfica:

In [ ]:
plot(title="Solución para x_1 con diversos valores de n usando RK_4",xlabel="t",ylabel="x1(t)")
for i in n 
    sol1=RK_4(sistema1,[0.0,1.0,cos(i*π/6),sin(i*π/6)],tiempo1)
    plot!(tiempo1,sol1[:,1],label ="n=$(i)", lw=1.0)
end
plot!()

In [ ]:
plot(title="Solución para x_2 con diversos valores de n usando RK_4",xlabel="t",ylabel="x2(t)")
for i in n 
    sol1=RK_4(sistema1,[0.0,1.0,cos(i*π/6),sin(i*π/6)],tiempo1)
    plot!(tiempo1,sol1[:,2],label ="n=$(i)", lw=1.0)
end
plot!()

In [ ]:
plot(title="Solución para x_3 con diversos valores de n usando RK_4",xlabel="t",ylabel="x3(t)")
for i in n 
    sol1=RK_4(sistema1,[0.0,1.0,cos(i*π/6),sin(i*π/6)],tiempo1)
    plot!(tiempo1,sol1[:,3],label ="n=$(i)", lw=1.0)
end
plot!()

In [ ]:
plot(title="Solución para x_4 con diversos valores de n usando RK_4",xlabel="t",ylabel="x4(t)")
for i in n 
    sol1=RK_4(sistema1,[0.0,1.0,cos(i*π/6),sin(i*π/6)],tiempo1)
    plot!(tiempo1,sol1[:,4],label ="n=$(i)", lw=1.0)
end
plot!()

#### (c) Genere una función que calcule la energía para las distintas condiciones iniciales y detenga la simulación cuando esta sea menor al 30 % de su valor inicial. Cabe mencionar que debe hacer su simulación (paso de integración) de tal manera que el error en la trayectoria no tenga variaciones de más del 1 %, es decir se debe asegurar convergencia a la solución con error de a lo mas 1 %.

La energía total de nuestro sistema será:
$$U=\frac{m}{2}(v_x^{2}+v_y^{2})+\frac{k}{2}(x^{2}+y^{2}) $$
Sustituimos k=1=m y se tiene que
$$U=\frac{1}{2}(v_x^{2}+v_y^{2}+x^{2}+y^{2}) $$
Así, definimos una función que nos de la energía del sistema a cada tiempo:

In [ ]:
function energia1(p0,t,EDO)
    x=zeros(length(t))
    y=zeros(length(t))
    vx=zeros(length(t))
    vy=zeros(length(t))
    x=RK_4(EDO,p0,t)[:,1]
    y=RK_4(EDO,p0,t)[:,2]
    vx=RK_4(EDO,p0,t)[:,3]
    vy=RK_4(EDO,p0,t)[:,4]
    #Definimos el arreglo donde guardaremos la información de la energía
    U=zeros(length(t))
    U[1]=0.5*(x[1]^2+y[1]^2+vx[1]^2+vy[1]^2)
    for i in 2:length(t)
        U[i]=0.5*(x[i]^2+y[i]^2+vx[i]^2+vy[i]^2)
        if U[i]>=0.3*U[1]
            println(U[i])
        end
    end
end

Obtengamos las variaciones de nuestra solución usando el porcentaje de diferencia:
$$dif\%=\frac{|x_{i+1}-x_{i}|}{|x_i|} $$

In [ ]:
function variacion(p0,t,EDO)
    #x=zeros(length(t))
    #y=zeros(length(t))
    #vx=zeros(length(t))
    #vy=zeros(length(t))
    #Definimos la difere
    x=RK_4(EDO,p0,t)[:,1]
    y=RK_4(EDO,p0,t)[:,2]
    vx=RK_4(EDO,p0,t)[:,3]
    vy=RK_4(EDO,p0,t)[:,4]
    difx=zeros(length(t))
    dify=zeros(length(t))
    difvx=zeros(length(t))
    difvy=zeros(length(t))
    for i in 2:length(t)
        if x[i-1]!=0
            difx[i]=abs((x[i]-x[i-1])/(x[i-1]))
        else
            difx[i]=-0
        end
        dify[i]=abs((y[i]-y[i-1])/(y[i-1]))
        difvx[i]=abs((vx[i]-vx[i-1])/(vx[i-1]))
        difvy[i]=abs((vy[i]-vy[i-1])/(vy[i-1]))
    end
    #mdifx=max(difx)
    return maximum(difx)
end

In [ ]:
tiempo1v=collect(0:0.01:0.01)
variacion([0.0,1.0,1.0,0.0],tiempo1v,sistema1)

In [ ]:
tiempo1c=collect(0:0.01:70π)
energia1([0.0,1.0,1.0,0.0],tiempo1c,sistema1)

#### (d) Dibuje las gráficas del momento angular para los casos anteirores en el inciso (1b). 

#### (e) Tome de manera arbitraria alguna condición inicial y dibuje el espacio fase (px, x).

#### 3. Se tiene la siguiente función: $$f(x,y)=2e^{(x+1)^{2}}\left(e^{-(y+2)^{2}}+e^{-(y-2)^{2}}\right)+6x(x^{2}+y^{2})e^{-(x^{2}+y^{2})}-1$$ Encuentre los puntos $(\~x, \~y)$ tales que $f(\~x, \~y)=0$ y dibuje estos puntos en el plano xy.
#### Hint: Como podrá notar los puntos a buscar se encuentran en la región x, y ∈ (−4, 4) × (−4, 4). Una forma de obtener los puntos es haciendo un barrido para x fija y encontrando todas las raíces de la ecuación en términos de y. Entre más fino haga el barrido, la curva de nivel que se encuentre será mejor.

In [ ]:
function raiz_NewtonMult(funcion,intervalo,delta,f_prima,error=1e-4)
    #Definimos el número de trozos en los que dividiremos nuestro sistema
    
    cortes=floor(Int,((intervalo[2]-intervalo[1])*delta))
    #Int((intervalo[2]-intervalo[1])*delta)
    #Definimos el arreglo donde anexaremos las raíces
    raices=[]
    iteraciones=[]
    #Hacemos un ciclo dónde se buscarán las raíces tomando como inicio puntos en ese espacio
    for i in 1:3
        #Integramos el método de Newton para cada i
        iteracion = 0
        #Definimos nuestro punto de inicio
        p_ini=intervalo[1]+i*delta
        while abs(funcion(p_ini)) > error
            p_ini += -(funcion(p_ini)/f_prima(p_ini))
            iteracion +=1
        end
        #Anexamos la raíz encontrada al arreglo raíces
        push!(raices,p_ini)
    end
    #Salimos del ciclo for y le pedimos a la función que nos regrese el arreglo de las raíces
    return raices
end

In [ ]:
raiz_NewtonMult(sin,[-1π,2π],0.1,cos)

In [8]:
function raiz_NewtonMult2(funcion,inicio,fin,delta,f_prima,error=1e-4)
    #Definimos el número de trozos en los que dividiremos nuestro sistema
    cortes=floor(Int,(fin-inicio)/delta)
    #Definimos el arreglo donde anexaremos las raíces
    raices=[]
    iteraciones=[]
    #Hacemos un ciclo dónde se buscarán las raíces tomando como inicio puntos en ese espacio
    for i in 0:cortes-1
        #Integramos el método de Newton para cada i
        iteracion = 0
        #Definimos nuestro punto de inicio
        p_ini=inicio+i*delta
        while abs(funcion(p_ini)) > error
            p_ini += -(funcion(p_ini)/f_prima(p_ini))
            iteracion +=1
        end
        #Anexamos la raíz encontrada al arreglo raíces
        push!(raices,p_ini)
    end
    #Salimos del ciclo for y le pedimos a la función que nos regrese el arreglo de las raíces
    return raices
end

raiz_NewtonMult2 (generic function with 2 methods)

In [16]:
function raiz_NewtonMult2filtro(funcion,inicio,fin,delta,f_prima,error=1e-4)
    #Definimos el número de trozos en los que dividiremos nuestro sistema
    cortes=floor(Int,(fin-inicio)/delta)
    #Definimos el arreglo donde anexaremos las raíces
    raices=[]
    iteraciones=[]
    #Hacemos un ciclo dónde se buscarán las raíces tomando como inicio puntos en ese espacio
    for i in 0:cortes-1
        #Integramos el método de Newton para cada i
        iteracion = 0
        #Definimos nuestro punto de inicio
        p_ini=inicio+i*delta
        while abs(funcion(p_ini)) > error
            p_ini += -(funcion(p_ini)/f_prima(p_ini))
            iteracion +=1
        end
        #Anexamos la raíz encontrada al arreglo raíces
        if p_ini==Inf
            false
        else
            push!(raices,p_ini)
        end
    end
    #Salimos del ciclo for y le pedimos a la función que nos regrese el arreglo de las raíces
    return raices
end

raiz_NewtonMult2filtro (generic function with 2 methods)

In [2]:
f(x,y=0)=2*exp(-(x+1)^2)*(exp(-(y+2)^2)+exp(-(y-2)^2)) + 6*x*(x^2 + y^2)*exp(-(x^2+y^2))-1
#df(x,y=0)=12*x^2*exp(-x^2-y^2)-12*x^2*exp(-x^2-y^2)*(x^2+y^2)+6*exp(-x^2-y^2)*(x^2+y^2)

f (generic function with 2 methods)

In [17]:
dfx(x,y=0)=-4*exp(-(x+1)^2)*(x+1)*(exp(-(y-2)^2)+exp(-(y+2)^2))+(exp(-x^2-y^2)*(18*x^2+6*y^2))-(12*x^2*(x^2+y^2)*exp(-(x^2+y^2)))

dfx (generic function with 2 methods)

In [19]:
raiz_NewtonMult2filtro(f,-4,4,1.5,dfx)

3-element Array{Any,1}:
 0.6260313908578603
 0.6260311571090578
 1.9469265118941532

In [22]:
function raiz_Newton(funcion,p_ini,f_prima,error=1e-4)
    iteracion = 0
    while abs(funcion(p_ini)) > error
        p_ini += -(funcion(p_ini)/f_prima(p_ini))
        iteracion +=1
    end
    return p_ini,iteracion
end

raiz_Newton (generic function with 2 methods)

In [24]:
raiz_Newton(f,-4,dfx)

(Inf, 2)

In [9]:
function raiz_bf(funcion,intervalo,error=1e-4)
    raiz = 0.5*(intervalo[1]+intervalo[2])
    iteracion = 0
    while abs(intervalo[2]-intervalo[1]) > error
        f_izq , f_med = funcion(intervalo[1]) , funcion(raiz)
        if f_izq*f_med < 0
            intervalo[2] = raiz
        else
            intervalo[1] = raiz
        end
        raiz = 0.5*(intervalo[1]+intervalo[2])
        iteracion += 1
    end
    return raiz,iteracion
end

raiz_bf (generic function with 2 methods)

In [14]:
raiz_bf(f,[0.9,3])

(1.946955871582031, 15)

In [ ]:
x = collect(-1π:0.1:2π)
plot(x,sin.(x),label="sin(x)",xlabel="x",ylabel="f(x)")
#raices=raiz_NewtonMult2(cos,-1π,2π,2,-sin)
#scatter!([raices[1]],[0],label="raíz")
#scatter!([raices[2]],[0],label="raíz")
#scatter!([raices[3]],[0],label="raíz")
#scatter!([raices[4]],[0],label="raíz")

In [ ]:
f(x,y)=2*exp((x+1)^2)*(exp(-(y+2)^2)+exp(-(y-2)^2))+6*x*(x^2+y^2)*exp(-(x^2+y^2))-1